In [1]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
import requests
import pandas as pd
import pickle
import os

In [2]:
chaves = {
    'a': 1,
    'b': 2
}

def soma(a, b):
    return a + b

In [3]:
chaves['a']

1

In [4]:
import pendulum

In [5]:
def obter_turno_pasta(hora: int) -> str: 
    if 0 >= hora < 6:
        turno = 'madrugada'
    elif 6 >= hora < 12:
        turno = 'manha'
    elif 12 >= hora < 18:
        turno = 'tarde'
    else:
        turno = 'noite'
    return turno

In [6]:
import pendulum
data_hora_atual = pendulum.now('America/Sao_Paulo').to_iso8601_string()
data_hora_atual = pendulum.parse(data_hora_atual)
hora = data_hora_atual.hour
hora

20

In [7]:
obter_turno_pasta(hora)

'noite'

In [8]:
data = data_hora_atual.format('YYYY_MM_DD')
data + '_' + str(obter_turno_pasta(hora))

'2024_07_09_noite'

In [9]:
data_hora_atual

DateTime(2024, 7, 9, 20, 15, 22, 342128, tzinfo=Timezone('-03:00'))

In [10]:
soma(**chaves)

3

In [11]:
spark = SparkSession.builder.appName('DEPARA').getOrCreate()

24/07/09 20:15:36 WARN Utils: Your hostname, rodrigo-nitroan51554 resolves to a loopback address: 127.0.1.1; using 192.168.3.6 instead (on interface wlp8s0)
24/07/09 20:15:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/07/09 20:15:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


- Estatísticas Canais

In [12]:
df_estatisticas_canais = spark.read.json(
    '../datalake/bronze/*/extracao_data_2024_07_09_tarde/estatisticas_canais_brasileiros/req_estatisticas_canais_brasileiros.json')

In [13]:
df_estatisticas_canais.printSchema()

root
 |-- assunto: string (nullable = true)
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- customUrl: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- localized: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- publishedAt: string (nullable = true)
 |    |    |    |-- thumbnails: struct (nullable = true)
 |    |    |    |    |-- default: struct (nullable = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |

In [14]:
def obter_turno(data_extracao_col: F.col):
    return F.when((F.hour(data_extracao_col) >= 0) & (F.hour(data_extracao_col) < 8), 'Manhã') \
            .when((F.hour(data_extracao_col) >= 8) & (F.hour(data_extracao_col) < 16), 'Tarde') \
            .otherwise('Noite')

In [15]:
df_estatisticas_canais = df_estatisticas_canais.select(
    'data_extracao',
    'assunto',
    F.explode('items').alias('items')
).select(
    F.col('assunto').alias('ASSUNTO'),
    F.col('data_extracao').alias('DATA_EXTRACAO'),
    F.hour('data_extracao').alias('ANO_EXTRACAO'),
    F.month('data_extracao').alias('MES_EXTRACAO'),
    F.dayofmonth('data_extracao').alias('DIA_EXTRACAO'),
    obter_turno(F.col('data_extracao')).alias('TURNO_EXTRACAO'),
    F.col('items.id').alias('ID_CANAL'),
    F.col('items.snippet.title').alias('NM_CANAL'),
    F.col('items.statistics.subscriberCount').alias('TOTAL_INSCRITOS'),
    F.col('items.statistics.videoCount').alias('TOTAL_VIDEOS_PUBLICADOS'),
    F.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES')
)
# df_estatisticas_canais = df_estatisticas_canais.withColumn('')
df_estatisticas_canais.show(truncate=False)

+---------------+-------------------+------------+------------+------------+--------------+------------------------+------------------------------------+---------------+-----------------------+-------------------+
|ASSUNTO        |DATA_EXTRACAO      |ANO_EXTRACAO|MES_EXTRACAO|DIA_EXTRACAO|TURNO_EXTRACAO|ID_CANAL                |NM_CANAL                            |TOTAL_INSCRITOS|TOTAL_VIDEOS_PUBLICADOS|TOTAL_VISUALIZACOES|
+---------------+-------------------+------------+------------+------------+--------------+------------------------+------------------------------------+---------------+-----------------------+-------------------+
|linux          |2024-07-09 15:10:31|15          |7           |9           |Tarde         |UCU2ar87kwe2ZkNKB8-ctVtQ|Objetivo Concursos                  |151000         |5366                   |11473628           |
|linux          |2024-07-09 15:10:31|15          |7           |9           |Tarde         |UCkh8ofkguG5Dta9cW_1T-9A|Liberdade e Harmonia com o U

In [16]:
df_estatisticas_canais_pandas = df_estatisticas_canais.toPandas()

In [17]:

# df_estatisticas_canais_pandas.to_parquet(
#     path='../datalake/prata/estatisticas_canais/estatisticas_canais_pandas.parquet', partition_cols=['TURNO_EXTRACAO'])

In [21]:
a = (1, 2)
print(*a)

1 2


In [18]:
import duckdb

con = duckdb.connect()

# Salvando o DataFrame em formato Parquet com partições
con.execute("""
    COPY (SELECT * FROM df_estatisticas_canais_pandas) 
    TO '{}' 
    (FORMAT PARQUET, PARTITION_BY ('TURNO_EXTRACAO'))
""".format('../datalake/prata/estatisticas_canais.parquet'))

con.close()

- Estatísticas Vídeos Brasileiros

In [16]:
df_estatisticas_videos = spark.read.json(
    '../datalake/bronze/*/extracao_data_2024_07_09_tarde/estatisticas_videos/req_estatisticas_videos.json')

In [17]:
df_estatisticas_videos.printSchema()

root
 |-- assunto: string (nullable = true)
 |-- data_extracao: string (nullable = true)
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- contentDetails: struct (nullable = true)
 |    |    |    |-- caption: string (nullable = true)
 |    |    |    |-- definition: string (nullable = true)
 |    |    |    |-- dimension: string (nullable = true)
 |    |    |    |-- duration: string (nullable = true)
 |    |    |    |-- licensedContent: boolean (nullable = true)
 |    |    |    |-- projection: string (nullable = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- categoryId: string (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- channelTitle: string (nullable = true)
 |    |    |    |-- defaultAudioLang

In [20]:
df_teste = df_estatisticas_videos.select('data_extracao', 'assunto', F.explode('items').alias('items')) \
    .select(
        F.col('assunto').alias('ASSUNTO'),
        F.col('data_extracao').alias('DATA_EXTRACAO'),
        F.hour('data_extracao').alias('ANO_EXTRACAO'),
        F.month('data_extracao').alias('MES_EXTRACAO'),
        F.dayofmonth('data_extracao').alias('DIA_EXTRACAO'),
        obter_turno(F.col('data_extracao')).alias('TURNO_EXTRACAO'),
        F.col('items.id').alias('ID_VIDEO'),
        F.col('items.snippet.channelId').alias('ID_CANAL'),
        F.col('items.snippet.title').alias('TITULO_VIDEO'),
        F.col('items.snippet.description').alias('DESCRICAO'),
        F.col('items.contentDetails.duration').alias('DURACAO'),
        F.col('items.snippet.tags').alias('TAGS'),
        
        F.col('items.snippet.categoryid').alias('ID_CATEGORIA'),
        F.col('items.statistics.viewCount').alias('TOTAL_VISUALIZACOES'),
        F.col('items.statistics.likeCount').alias('TOTAL_LIKES'),
        F.col('items.statistics.favoriteCount').alias('TOTAL_FAVORITOS'),
    )
df_teste = df_teste.withColumn('TOTAL_TAGS', F.when(
    F.size(df_teste.TAGS) <= 0, 0).otherwise(F.size(df_teste.TAGS)))
df_teste = df_teste.withColumn('TOTAL_PALAVRAS_TITULO', F.size(F.split(df_teste.TITULO_VIDEO, " ")))
df_teste = df_teste.withColumn(
    'TOTAL_PALAVRAS_DESCRICAO', F.size(F.split(df_teste.DESCRICAO, " ")))
df_teste.show(truncate=False)

+---------------+-------------------+------------+------------+------------+--------------+-----------+------------------------+--------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
spark.stop()